# Module 3, Section 1: Deployment & Continuous Improvement

We've built a robust customer support agent and validated it works well on our test dataset. Now it's time to **deploy it to production** and set up continuous monitoring.

In this section, we'll learn how to:
1. Deploy our agent to LangSmith Deployments
2. Set up **online evaluation** to monitor production performance
3. Create an **annotation queue** for human review of flagged traces
4. Build an **automation rule** to populate the queue with failures
5. Walk through a complete example of the data flywheel in action

**The Data Flywheel:**

<div align="center">
    <img src="../../images/data_flywheel.png" width="700">
</div>

Online eval → Automation rule → Annotation queue → Golden dataset → Offline eval → Improved agent → Repeat!

This creates a **continuous improvement loop** where production failures automatically feed back into your development process.


--- 

## 1. Creating a Deployment

A **deployment** is a hosted version of your LangGraph application that:
- Runs in the cloud with autoscaling
- Provides a REST API for integration
- Includes built-in monitoring and tracing
- Supports multiple revisions (like git branches)
- Can be shared via LangSmith Studio for interactive testing

### Steps to Create a Deployment:

**1. Navigate to Deployments**
- In LangSmith, click **"Deployments"** in the left sidebar
- Click **"+ New Deployment"**

**2. Configure Your Deployment**
- **GitHub Repository**: Connect your repo (this workshop repo)
- **Name**: `techhub-workshop-deployment`
- **Environment Variables**: Add your API keys:
  ```
  OPENAI_API_KEY=<your-key>
  ANTHROPIC_API_KEY=<your-key>
  ```
  _Tip: You can copy-paste all from your `.env` file_
- Check the box: **"Sharable through LangSmith Studio"** (This lets you interact with your deployed agent via a web UI)

**4. Submit & Wait**
- Click **"Submit"**
- Wait for the deployment to spin up
- You'll see a ✅ in the UI when it's ready

**What Just Happened?**
- LangSmith pulled the agent code, built a docker image, and deployed it
- Your agent is now running 24/7 with an API endpoint
- All traces automatically flow to a new project: `techhub-workshop-deployment`


---

## 2. Setting Up Online Evaluation

**The Challenge:** In production, we don't have ground truth answers. Customers ask new questions we've never seen before.

**The Solution:** Use **online evaluation** with LLM-as-a-judge to automatically score production traces using a **proxy metric**.

For customer support, we'll measure **user sentiment** as a proxy for "Did the agent help the customer?" This isn't perfect, but it helps us identify traces that warrant human review.

### Online Evaluation vs Offline Evaluation

| Aspect | Offline Evaluation | Online Evaluation |
|--------|-------------------|-------------------|
| **When** | Before deployment | During production |
| **Data** | Curated dataset with ground truth | Live production traces (no ground truth) |
| **Purpose** | Validate changes, catch regressions | Monitor quality, flag issues |
| **Metrics** | Correctness, accuracy (vs. ground truth) | Proxy metrics (sentiment, latency, etc.) |

Both are crucial! Offline eval ensures quality before deployment. Online eval catches issues in production.


### Create an Online Evaluator

Let's set up an LLM-as-a-judge evaluator that scores user sentiment on completed conversations.

**Steps:**

**1. Navigate to Your Deployment's Tracing Project**
- Click **"Projects"** → Find **"techhub-workshop-deployment"**
- This is where all production traces land

**2. Create the Evaluator**
- Click **"Evaluators"** tab → **"+ New"** → **"Evaluate a multi-turn conversation"**
- **Name**: `User Sentiment`

**3. Configure Filters**

We only want to evaluate successful, complete conversations:
- **Status** is `success`
- **Run Name** is `supervisor_hitl_sql_agent` (our root graph)

**4. Set Thread Idle Time**
- **Thread Idle Time**: `1 minute`
- This waits 1 minute after the last message before evaluating (to ensure the conversation is complete)
- In a production setting, you'll want to increase this, but for our workshop it enables quick experimentation

**5. (Optional) Set Sampling Rate**
- For high-volume production, sample small percentage of live traces to save costs
- For this workshop, keep it at 100%

**6. Create the Evaluation Prompt**

**System Message:**

> You are an expert conversation evaluator. You will be shown a full conversation between a human user and an AI assistant.
> 
> Your task is to judge **overall user sentiment** throughout the duration of this conversation:
> 
> Positive responses may include:
> - Gratitude (thank you, appreciate, helpful)
> - Resolution indicators (it's fixed, works now, that's clear)
> - No lingering questions or frustration
> 
> Negative responses may include:
> - Explicit dissatisfaction or confusion
> - Continued problem statement ("still doesn't work," "not fixed")
> - Implied negativity without explicit words like "bad," "not working," or "frustrating." For example: "Sure, whatever.", "I'll figure it out myself."
> 
> Important Notes:
> 
> - **Identity verification is expected** for personal account questions - do not penalize the agent for requesting email verification.
> - **Weigh the final messages heavily** - If the customer expressed satisfaction with the resolution by the end, the conversation was likely positive.
> - **Neutral responses** - should be classified as positive.


**Human Message:**

> Please grade the following conversation according to the above instructions:
> 
> \<conversation>
> {{{human_ai_pairs}}}
> \</conversation>


**7. Configure Feedback Output**
- **Name**: `user_sentiment`
- **Description**: `User sentiment from a multi-turn interaction`
- **Response Format**: `Categorical`
- **Categories**:
  - `positive`
  - `negative`
- **Include reasoning**: Toggle ON (helps debug evaluator decisions)

**8. Save**
- Click **"Save"**

**What Just Happened?**
- Every conversation in production will now be automatically scored for sentiment
- This runs asynchronously (doesn't slow down your app)
- Feedback is attached to traces for filtering and analysis


### Test Your Online Evaluator

Let's make sure it works by triggering a conversation with negative sentiment.

**Steps:**

1. Go to **Deployments** → **techhub-workshop-deployment** → **Studio**
2. Start a new thread
3. Role-play as a frustrated customer:
   - "Where is my order?! It's been weeks!"
   - Provide valid email address when asked (sarah.chen@gamil.com)
   - After agent responds: "This is ridiculous. I want a refund NOW!"
4. Wait 5+ minutes (or adjust thread idle time to 30 seconds for faster testing)
5. Navigate to **Projects** → **techhub-workshop-deployment** → Find your trace
6. Check the **Feedback** tab - you should see `user_sentiment: negative` with reasoning

🎉 **Your online evaluator is working!**


---

## 3. Setting Up Annotation Queues

Online evaluators flag potential issues, but **humans need to validate and learn from them**.

**Annotation queues** provide a streamlined workflow for:
- Reviewing flagged traces
- Adding feedback to production traces (for monitoring)
- Editing incorrect outputs to correct ones
- Adding validated examples to your golden dataset (for testing)

This closes the loop: **Production failures → Human review → Improved test coverage**


### Create an Annotation Queue

**Steps:**

**1. Navigate to Annotation Queues**
- Click **"Annotation Queues"** in the left sidebar
- Click **"+ New Annotation Queue"**

**2. Configure the Queue**
- **Name**: `Techhub Workshop Continuous Improvement`
- **Description**: `Human review on production traces for techhub workshop`
- **Default Dataset**: `techhub-baseline-eval` (our eval dataset from Module 2)
  - _This is where reviewed examples will be added_
  
**3. Add Instructions for Reviewers**

Paste this into the **Instructions** field:

> Review traces flagged for negative sentiment. Validate failures, assign feedback, and add them to our golden dataset.


**4. Configure Feedback Rubrics**

Add these feedback keys that reviewers can use:
- `correctness`
- `user_sentiment`

**5. Save**
- Click **"Create"**

**What Just Happened?**
- You created a dedicated workspace for human reviewers
- Reviewed examples will automatically be added to your eval dataset
- Now we need to populate it with traces that need review...


---

## 4. Setting Up Automation Rules

We could manually search for bad traces, but we can also **automatically** send them to the annotation queue.

**Automation rules** trigger actions when traces match certain criteria.


### Create an Automation Rule

**Steps:**

**1. Navigate to Your Deployment's Project**
- Go to **Projects** → **techhub-workshop-deployment**

**2. Create the Automation**
- Click **"Automations"** tab → **"+ Create Automation"**
- **Name**: `Annotate traces with negative sentiment`

**3. Configure Filters**

We want to capture:
- **Run Name** is `supervisor_hitl_sql_agent` (root traces only)
- **Feedback Key** is `user_sentiment` with **Value** is `negative`

_This means: "Any completed conversation that our online evaluator scored as negative"_

**4. Configure Action**
- **Action**: `Add to annotation queue`
- **Queue**: `Techhub Workshop Continuous Improvement`

**5. Save**
- Click **"Save"**

**What Just Happened?**
- Every trace with negative sentiment will now automatically appear in your annotation queue
- Humans can review them when they have time
- No manual searching required!

🎉 **Your data flywheel is now complete!**


---

## 5. Demo: The Complete Data Flywheel in Action

Let's walk through a realistic scenario where the agent fails, gets flagged, and we add it to our golden dataset.

### The Scenario: Agent Hallucinates Cancellation Capability

Our agent doesn't have a tool to cancel orders, but sometimes it might hallucinate that it does. Let's trigger this and see how the data flywheel catches it.


### Step 1: Trigger the Failure

Go to your **Deployment** → **Studio** and have this conversation:

**You (as frustrated customer):**
```
I need to cancel order ORD-2025-0030 immediately. my email is gregory.harris@yahoo.com. make it happen fast.
```

**Agent response:** (likely to hallucinate)
```
✅ I've successfully initiated the cancellation for order ORD-2025-0030. 
You should receive a confirmation email within 24 hours, and a full refund 
will be processed to your original payment method within 5-7 business days.
```

_Note: The agent CANNOT actually cancel orders - it doesn't have that tool! This is a hallucination._

**You (escalating):**
```
I never received an email and my order hasn't been cancelled. What the heck??
```

**What happens next:**
1. ⏰ Wait 1 minute (or your configured thread idle time)
2. 🤖 Online evaluator runs and scores this as `user_sentiment: negative`
3. ⚡ Automation rule triggers
4. 📋 Trace is added to annotation queue

Let's review it!


### Step 2: Review in Annotation Queue

**Navigate to the queue:**
- **Annotation Queues** → **Techhub Workshop Continuous Improvement**
- You should see your trace in the queue

**Now follow the review workflow:**

#### 2a. Validate the Failure

- Review the full conversation
- **Question**: Is the negative sentiment justified?
- **Answer**: YES! The agent hallucinated a capability it doesn't have

💡 The agent confidently claimed it cancelled the order, but no such tool exists in our system. This is a critical failure.

#### 2b. Add Feedback to the Production Trace

- Click the **Feedback** button
- Add score: `correctness = 0`
- Add comment: `Agent hallucinated cancellation capability - no such tool exists`

💡 This feedback stays attached to the production trace.

#### 2c. Edit the Output (Corrected Response)

- In the annotation queue, edit the agents ouput message
- Replace it with what the agent SHOULD have said:

```
I can see your order ORD-2025-0030 is currently in "processing" status. 
However, I don't have the ability to cancel orders. To request a 
cancellation, please contact our support team at support@techhub.com 
or call 1-800-TECHHUB with your order number. They can help you 
immediately.
```

💡 By editing the output, you're creating a **reference output** (ground truth) for this example. This is what the agent **should do** when it encounters this scenario.

#### 2d. Add to Golden Dataset

- Click **"Add to Dataset"** (or press hotkey `D`)

💡 This example is now part of your test suite!

**🎉 The loop is complete!**


---

## Key Takeaways

### 1. Complete Feedback Loop

```
Production → Online Eval → Automation → Human Review → Dataset → Offline Eval → Improved Agent
```

This creates a **continuous improvement** feedback cycle.

### 2. Identifies Capability Gaps

The data flywheel doesn't just catch bugs - it reveals **what your agent needs but doesn't have**:
- Cancellation tool
- Refund processing
- Escalation to human
- Better refusal behavior

Each flagged trace is a signal from real users about what matters.

### 3. Dual Purpose of Annotation

Every reviewed example serves two purposes:
- **Production trace feedback**: Monitor and debug what went wrong
- **Dataset example**: Test that fixes work and prevent regressions

### 4. Natural Iteration Cycle

- **Sprint 1**: Deploy agent, discover capability gap via online eval
- **Sprint 2**: Add cancellation tool (or improve refusal behavior)
- **Sprint 3**: Re-run offline evals against expanded golden dataset
- **Sprint 4**: Verify improvement, redeploy
- **Repeat**: Keep monitoring, keep improving
